In [4]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS, NET_TIMEOUT_SECONDS, SUMO_TOTAL_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType, BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS, ROOT_DIR
from od.env.station import BS_PRESET
from od.social.manager.types import DynamicSocialGroupBehaviour
import od.misc.logger
import od.misc.statistic as oms
from datetime import datetime


In [5]:
class FakeBaseStation:
    def __init__(self, name, pos, bs_type, serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial

    def __repr__(self) -> str:
        return self.name

    def __index__(self):
        return self.serial


In [9]:
def Run(statistic, interest_config):
    statistic.nft_traffic = {}
    statistic.raw_sg_header = None
    statistic.interest_config = interest_config
    statistic.dirpath = "../data/GERNAS/POST_GERNA/" + statistic.interest_config.folder()
    print(statistic.dirpath)
    BS_CONFIG = {
        name: bs_config
        for name, bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA or
            (
                interest_config.req_rsu and
                bs_config["type"] == BaseStationType.UMI
            )
        )
    }
    time_text = datetime.now().strftime("%Y-%m-%d %H-%M-%S")

    class GV:
        NET_STATION_CONTROLLER = [
            FakeBaseStation(
                name,
                BS_CONFIG[name]["pos"],
                BS_CONFIG[name]["type"],
                i
            ) for i, name in enumerate(BS_CONFIG)
        ]
        STATISTIC = od.misc.logger.Logger(statistic.dirpath, "Statistic ({}).txt".format(time_text))
        RESULT = od.misc.logger.Logger(statistic.dirpath, "Result ({}).txt".format(time_text))
    oms.GV = GV
#############################################################################################################
    # preprocess overtimes
    for sg in statistic.social_group:
        for record in statistic.sg_header[sg].values():
            for bs in GV.NET_STATION_CONTROLLER:
                if(len(record.time_bs_txq[bs]) > 0 and record.time_bs_txq[bs][-1][0] == SUMO_TOTAL_SECONDS and record.time_bs_txq[bs][-1][1] == SUMO_TOTAL_SECONDS and record.time_bs_drop[bs] == SUMO_TOTAL_SECONDS):
                    record.time_bs_drop[bs] = -1
                    record.time_bs_txq[bs].pop()
                    record.time_bs_txq[bs].append([record.time_bs_tx[bs][-1][1], SUMO_TOTAL_SECONDS + 1])
                if(len(record.time_bs_txq[bs]) > 0 and record.time_bs_txq[bs][-1][1] == SUMO_TOTAL_SECONDS):
                    record.time_bs_txq[bs][-1][1] = SUMO_TOTAL_SECONDS + 1
    c2g_app_stats = statistic.ExtractNetworkTraffic(NetFlowType.C2G, True)
    cri_app_stats = statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL, True)
    for name in c2g_app_stats.keys():
        c2g_app_stats[name].time_veh_serv = cri_app_stats[name].time_veh_serv
        c2g_app_stats[name].time_veh_tx = cri_app_stats[name].time_veh_tx
        c2g_app_stats[name].time_veh_txq = cri_app_stats[name].time_veh_txq
    with open(statistic.dirpath + "/object.pickle", "wb") as new_obj_file:
        pickle.dump(statistic, new_obj_file)
#############################################################################################################
    statistic.Report(True)
    GV.STATISTIC.Encapsulate()
    GV.RESULT.Encapsulate()


In [7]:
interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20, True, ResourceAllocatorType.NOMA_OPT, True,1.0, 6)
old_statistic = None
with open("../data/GERNAS/GERNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
    old_statistic = pickle.load(file)
Run(old_statistic, interest_config)

../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/yQoS/yRSU/NOMA_OPT/D1.0/S6/
Skipping Preprocess...


In [10]:
for res_alloc_type in [ResourceAllocatorType.NOMA_APR,ResourceAllocatorType.NOMA_OPT]:
    for rsu in [False, True]:
        for qos_re_class in [True, False] if rsu else [False]:
            for traffic_scale in [i / 10 for i in range(10, 15, 1)]:
                for seed in [i + 1 for i in range(10)]:
                    interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20, qos_re_class, res_alloc_type, rsu, traffic_scale, seed)
                    old_statistic = None
                    with open("../data/GERNAS/GERNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
                        old_statistic = pickle.load(file)
                    Run(old_statistic, interest_config)


../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S1/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S2/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S3/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S4/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S5/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S6/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S7/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S8/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S9/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S10/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_A

In [11]:
for dyn_sg_behav in DynamicSocialGroupBehaviour:
    for dyn_sg_conf in [(i*2+5) for i in range(5)]:
        for qos_re_class in [True ,False]:
            for seed in [i + 1 for i in range(10)]:        
                interest_config = InterestConfig(dyn_sg_behav,dyn_sg_conf, qos_re_class, ResourceAllocatorType.NOMA_APR, True, 1.4, seed)
                old_statistic = None
                with open("../data/GERNAS/GERNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
                    old_statistic = pickle.load(file)
                Run(old_statistic, interest_config)

../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S1/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S2/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S3/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S4/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S5/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S6/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S7/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S8/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S9/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/yQoS/yRSU/NOMA_APR/D1.4/S10/
Skipping Preprocess...
../data/GERNAS/POST_GERNA/MAX_N_MEMBER(5)/nQoS/yRSU/NOMA_APR/D1.4/S1/